In [1]:
import plotly.graph_objects as go
import numpy as np
import random
import copy
import time

Other funtions:

[https://www.sfu.ca/~ssurjano/optimization.html](https://www.sfu.ca/~ssurjano/optimization.html)

## **Class implementation**

In [2]:
# only one global minimum (-3.77, -3.28)
def himmelblau_aux(chromosome: list[float, float]) -> float:
	x = chromosome[0]
	y = chromosome[1]
	fxy = (x**2 + y - 11)**2 + (x + y**2 -7)**2
	if x>0 or y>0: 
		fxy += 0.5
	return fxy

# fitness para himmelblau: valor mínimo de la función
def fitness_himmel(chromosome: list[float, float]):
	return 1 / (1 + himmelblau_aux(chromosome))

In [3]:
class AG_Himmelblau():
    def __init__(self, N: int):
        self.population = [[random.uniform(-5, 5) for _ in range(2)] for _ in range(N)]
        self.fitnesses = []

    def sort_pop(self, fitness_function, reverse_sort: bool) -> tuple[list[list], list]:
        """Sort population by fitness function. Return tuple with population list and fitness list"""

        fitness_list = [fitness_function(ind) for ind in self.population]
        lista = sorted(zip(self.population, fitness_list), key=lambda x: x[1], reverse=reverse_sort)
        self.population = [x[0] for x in lista]
        self.fitnesses = [x[1] for x in lista]

    def select(self, T: int) -> list[float, float]:
        """Return a copy of an indivudual by tournament selection. Population already ordered by fitness"""

        choices=random.choices(copy.copy(self.population), k=T)
        indices=[self.population.index(c) for c in choices]
        return self.population[np.argmin(indices)]

    def crossover(self, parent1: list[float, float], parent2: list[float, float], pcross: float) -> tuple[list,list]: 
        if random.random()<pcross:
            child1 = [parent1[0], parent2[1]]
            child2 = [parent1[1], parent2[0]]
        else:
            child1, child2 = parent1[:], parent2[:]
        return child1, child2

    def mutate(self, individual: list[float, float], pmut: float) -> list[float, float]:
        if random.random()<pmut:
            individual[0], individual[1] = [random.uniform(-5, 5) for _ in range(2)]
        return individual

    def evolve(self, fitness_function, pmut=0.1, pcross=0.7, ngen=100, T=2, trace=50, reverse_sort=False, elitism=False) -> None:
        """Evolution procedure. Initial population already created"""

        for i in range(ngen):
            new_pop = []
            self.sort_pop(fitness_function, reverse_sort)
            if elitism:
                new_pop.append(self.population[0])
                new_pop.append(self.population[1])
            while len(new_pop) != 100:   
                individual1 = self.select(T)
                individual2 = self.select(T)
                child1, child2 = self.crossover(individual1, individual2, pcross)
                mutated1 = self.mutate(child1, pmut)
                mutated2 = self.mutate(child2, pmut)
                new_pop.append(mutated1)
                new_pop.append(mutated2)
                
            self.population = [*new_pop] # make a copy

            if i % trace == 0 or i == ngen-1: # en la última gen se ordena
                self.sort_pop(fitness_function, reverse_sort)
                print(f"Nº gen: {i}, Best ind: {self.population[0]}, Best fitness: {self.fitnesses[0]:.3f}")

## **Visual test**

In [6]:
def himmelblau_test(x, y):
    return (x**2 + y - 11)**2 + (x + y**2 - 7)**2

def plot_himmelblau(solution: list[float,float]):
    # grid of points
    x = np.linspace(-5, 5, 100)
    y = np.linspace(-5, 5, 100)
    X, Y = np.meshgrid(x, y)
    Z = himmelblau_test(X, Y)

    # Create the surface plot
    fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='Jet', showscale=False, opacity=0.8)])

    global_minima = [(3, 2), (-2.80, 3.13), (-3.77, -3.28), (3.58, -1.84)]
    for i, minimum in enumerate(global_minima): # plot minimum points
        x, y = minimum
        z = himmelblau_test(x, y)
        color = "purple" if i == 2 else "red"
        fig.add_trace(go.Scatter3d(
            x=[x], y=[y], z=[z],
            mode='markers',
            marker=dict(size=6, color=color, symbol='circle'),
            name=f'Minimum ({x}, {y})'
        ))

    # solution
    sol_x, sol_y = solution
    sol_z = himmelblau_test(sol_x, sol_y)  # Compute the Z-value for the solution
    fig.add_trace(go.Scatter3d(
        x=[sol_x], y=[sol_y], z=[sol_z],
        mode='markers',
        marker=dict(size=6, color="black", symbol='circle'),
        name=f'Solution ({sol_x:.3f}, {sol_y:.3f})'
    ))

    # Update layout
    fig.update_layout(
        title='Himmelblau Function',
        scene=dict(
            xaxis_title='X-axis',
            yaxis_title='Y-axis',
            zaxis_title='Z-axis'
        ),
        margin=dict(l=0, r=0, b=0, t=40),  # Adjust margins for better fit
        width=700,
        height=550,
        showlegend=True,
        legend=dict(
            x=1,
            y=1,
            xanchor='right',
            yanchor='top'
        )
    )

    fig.show()
# -----------------------------------------------
plot_himmelblau([1.0, 2.0])

## **---------------------------Tests---------------------------**

In [8]:
start = time.time()
genetic_algorithm = AG_Himmelblau(N=100) 
genetic_algorithm.evolve(fitness_function=fitness_himmel, pmut=0.1, ngen=1000, T=6, trace=100, reverse_sort=True)
minutos, segundos = divmod(time.time()-start, 60)
print(f"*******Tiempo transcurrido: {int(minutos)} minutos y {segundos:.2f} segundos*******")

plot_himmelblau(genetic_algorithm.population[0])

Nº gen: 0, Best ind: [3.1169168878397695, 1.6265137983757763], Best fitness: 0.318
Nº gen: 100, Best ind: [-3.875371140540018, -3.3543326673963483], Best fitness: 0.632
Nº gen: 200, Best ind: [-3.875371140540018, -3.3543326673963483], Best fitness: 0.632
Nº gen: 300, Best ind: [3.039839827584583, 2.0038068330576317], Best fitness: 0.640
Nº gen: 400, Best ind: [-3.7860523779852473, -3.2867571254892414], Best fitness: 0.997
Nº gen: 500, Best ind: [-3.7860523779852473, -3.2867571254892414], Best fitness: 0.997
Nº gen: 600, Best ind: [-3.7860523779852473, -3.2867571254892414], Best fitness: 0.997
Nº gen: 700, Best ind: [-3.7860523779852473, -3.2867571254892414], Best fitness: 0.997
Nº gen: 800, Best ind: [-3.7860523779852473, -3.2867571254892414], Best fitness: 0.997
Nº gen: 900, Best ind: [-3.7860523779852473, -3.2867571254892414], Best fitness: 0.997
Nº gen: 999, Best ind: [-3.7860523779852473, -3.2867571254892414], Best fitness: 0.997
*******Tiempo transcurrido: 0 minutos y 0.66 segundo